In [9]:
import os 
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [2]:
from transformers import BertModel

In [3]:
import torch
from kobert_transformers import get_kobert_model
from kobert_transformers import get_tokenizer

In [4]:
tokenizer = get_tokenizer()
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [5]:
import sqlite3

In [38]:
con = sqlite3.connect('/repo/TabularSemanticParsing/data/wikisql1.1/dev.db')

In [39]:
cur = con.cursor()

In [42]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")

In [43]:
print(cur.fetchall())

41438_1',), ('table_2_1604842_1',), ('table_2_1604940_12',), ('table_2_16050349_10',), ('table_2_16050349_14',), ('table_2_16050349_6',), ('table_2_160510_5',), ('table_2_16056990_12',), ('table_2_16066729_1',), ('table_2_16067721_1',), ('table_2_16070554_1',), ('table_2_16078390_1',), ('table_2_1608306_4',), ('table_2_16090138_1',), ('table_2_160994_2',), ('table_2_16100029_2',), ('table_2_16100029_3',), ('table_2_1610301_1',), ('table_2_1615758_2',), ('table_2_16181680_3',), ('table_2_16183862_1',), ('table_2_16185580_1',), ('table_2_1618638_2',), ('table_2_16194551_5',), ('table_2_161972_2',), ('table_2_161972_6',), ('table_2_16215078_1',), ('table_2_162342_2',), ('table_2_16234974_1',), ('table_2_16236714_2',), ('table_2_1625631_1',), ('table_2_1625862_8',), ('table_2_16270492_2',), ('table_2_16275569_1',), ('table_2_16279834_3',), ('table_2_1628307_5',), ('table_2_16285899_1',), ('table_2_1629086_5',), ('table_2_16292316_4',), ('table_2_16293403_29',), ('table_2_16295105_1',), ('t

In [46]:
for row in cur.execute('SELECT * FROM table_2_17231267_1'):
    print(row)

('johnny miller', 'united states', '1973', 282.0, 2.0, 't4')
('hale irwin', 'united states', '1974 , 1979', 284.0, 4.0, '6')
('lee trevino', 'united states', '1968 , 1971', 286.0, 6.0, 't9')
('tom watson', 'united states', '1982', 287.0, 7.0, 't11')
('david graham', 'australia', '1981', 287.0, 7.0, 't11')
('jack nicklaus', 'united states', '1962 , 1967 , 1972 , 1980', 289.0, 9.0, 't21')
('hubert green', 'united states', '1977', 291.0, 11.0, 't30')
('gary player', 'south africa', '1965', 294.0, 14.0, 't43')


In [47]:
from pypapago import Translator


ModuleNotFoundError: No module named 'pypapago'

In [49]:
from pypapago import Translator

In [50]:
translator = Translator()

In [54]:
result = translator.bulk_translate(['apple', 'banana'])


KeyError: 'translatedText'

In [55]:
import base64
import re
import requests
from multiprocessing import Pool, cpu_count
from functools import partial
from tqdm.auto import tqdm


class Translator:
    """
    Main Translator Class
    """

    def __init__(self, regex_pattern=None, headers=None):
        self.regex_pattern = re.compile(regex_pattern or '[가-힣]+')
        self.headers = headers or {
            'device-type': 'pc',
            'origin': 'https://papago.naver.com',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'ko',
            'authority': 'papago.naver.com',
            'pragma': 'no-cache',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko)\
                           Chrome/75.0.3770.100 Safari/537.36',
            'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'accept': 'application/json',
            'cache-control': 'no-cache',
            'x-apigw-partnerid': 'papago',
            'referer': 'https://papago.naver.com/',
            'dnt': '1',
        }
        self.SECRET_KEY = 'MxnUvVuGj2'
        self.QUERY_KEY = '0,"honorific":false,"instant":false,"source":"{source}","target":"{target}","text":"{query}"}}'

    @staticmethod
    def string_to_base64(s):
        """
        Generate Base64 Encoded string
        :param s: Origin Text (UTF-8)
        :return: B64 encoded text (B64, still UTF-8 string)
        """
        return base64.b64encode(s.encode('utf-8')).decode('utf-8')

    def translate(self, query, source='en', target='ko', verbose=False):
        """
        Main Translate function
        :param query: Original Text to translate
        :param source: Source(Original) text language [en, ko]
        :param target: Target text language [en, ko]
        :param verbose: Return verbose json data. Default: False
        :return: Translated text
        """
        data = {
            'data': self.SECRET_KEY + self.string_to_base64(
                self.QUERY_KEY.format(source=source, target=target, query=query)
            )
        }
        response = requests.post('https://papago.naver.com/apis/n2mt/translate', headers=self.headers, data=data)
        if not verbose:
            return response.json()['translatedText']
        return response.json()

    def bulk_translate(self, queries, source='en', target='ko', workers=None, verbose=False):
        """
        Call Translate function in parallel
        :param queries: List of query texts
        :param source: Source(Original) text language [en, ko]
        :param target: Target text language [en, ko]
        :param workers: Python multiprocessing workers. Default: vCPU cores
        :param verbose: Return verbose json data. Default: False
        :return: List of translated texts
        """
        with Pool(workers or cpu_count()) as pool:
            result = list(tqdm(pool.imap(
                func=partial(self.translate, source=source, target=target, verbose=verbose),
                iterable=queries
            ), total=len(queries)))
            pool.close()
            pool.join()
            return result

In [56]:
translator = Translator()

result = translator.bulk_translate(
   ['카카오는 파파고를 좋아해'],
    source='ko',
    target='en',
)
print(result) # Kakao likes papago.

KeyError: 'translatedText'